<a href="https://colab.research.google.com/github/ynyonato/HandsOn/blob/updates/ia_me_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import des librairies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob

In [ ]:
# Télécharger stopwords et autres ressources NLTK si besoin
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
# Chargement des données (ici on suppose que le fichier csv est uploadé dans Colab)
df = pd.read_csv('ia-me/suivi_activites.csv', sep=';', encoding='latin-1', quotechar='"')
print(df.columns)
print(df.head())

In [ ]:
# Nettoyage basique du texte (feedback)
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zàâçéèêëîïôûùüÿñæœ\s]", '', text)  # enlever ponctuations sauf lettres accentuées
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['Feedback_clean'] = df['Feedback'].fillna('').apply(clean_text)
# print(df['Feedback_clean'].head())

In [ ]:
# Suppression des stopwords
stop_words = set(stopwords.words('french'))
def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stop_words])

df['Feedback_clean'] = df['Feedback_clean'].apply(remove_stopwords)
# print(df['Feedback_clean'].head())

In [ ]:
# Check if there are any non-empty strings after cleaning and stopword removal
if df['Feedback_clean'].str.strip().eq('').all():
    print("Erreur: Après nettoyage et suppression des mots vides, la colonne 'Feedback_clean' est vide. Impossible de créer un vocabulaire TF-IDF.")
    # Option 1: Print some raw feedback to understand why it's empty
    print("\nExemples de feedback originaux :")
    print(df['Feedback'].head())
    # Option 2: You might need to revisit your cleaning steps or data source
else:
    vectorizer = TfidfVectorizer(max_features=20)
    X = vectorizer.fit_transform(df['Feedback_clean'])
    tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

    # Somme des TF-IDF par mot pour voir les plus importants
    tfidf_sum = tfidf_df.sum().sort_values(ascending=False)
    print("Top mots-clés TF-IDF :")
    print(tfidf_sum)

In [ ]:
X = vectorizer.fit_transform(df['Feedback_clean'])
tfidf_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

In [ ]:
# Somme des TF-IDF par mot pour voir les plus importants
tfidf_sum = tfidf_df.sum().sort_values(ascending=False)
print("Top mots-clés TF-IDF :")
print(tfidf_sum)

In [ ]:
# Analyse de sentiment avec TextBlob (en français)
# On utilise TextBlob directement ici, qui marche mieux en anglais, mais ça donnera un aperçu
# Pour un vrai traitement en français, on pourrait utiliser d’autres libs (ex: HuggingFace)
def sentiment_polarity(text):
    return TextBlob(text).sentiment.polarity

df['Sentiment'] = df['Feedback_clean'].apply(sentiment_polarity)

In [ ]:
# Catégorisation simple du sentiment
def categorize_sentiment(score):
    if score > 0.1:
        return 'Positif'
    elif score < -0.1:
        return 'Négatif'
    else:
        return 'Neutre'

df['Sentiment_cat'] = df['Sentiment'].apply(categorize_sentiment)

In [ ]:
# Affichage des counts sentiment
print("\nRépartition des sentiments :")
print(df['Sentiment_cat'].value_counts())

In [ ]:
# Nuage de mots pour feedback global
all_text = ' '.join(df['Feedback_clean'])
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_text)

plt.figure(figsize=(12,6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("Nuage de mots - Feedback")
plt.show()

In [ ]:
# Graphique des sentiments par type d'activité
plt.figure(figsize=(10,6))
sns.countplot(data=df, x='Type_Activite', hue='Sentiment_cat')
plt.title("Sentiments par Type d'Activité")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Moyenne de sentiment par localisation
sentiment_localisation = df.groupby('Localisation')['Sentiment'].mean().sort_values()

plt.figure(figsize=(8,5))
sentiment_localisation.plot(kind='bar', color='skyblue')
plt.title("Moyenne du sentiment par localisation")
plt.ylabel("Score moyen de sentiment")
plt.show()

In [ ]:
# Croisement nombre participants vs sentiment (boxplot)
plt.figure(figsize=(8,5))
sns.boxplot(data=df, x='Sentiment_cat', y='Nombre_Participants')
plt.title("Nombre de participants selon sentiment")
plt.show()